In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import Ridge
import pandas as pd
from scipy.sparse import hstack

In [21]:
# Загрузите данные об описаниях вакансий и соответствующих 
# годовых зарплатах из файла salary-train.csv.
data_train = pd.read_csv('salary-train.csv')
data_test = pd.read_csv('salary-test-mini.csv')

In [32]:
# Проведите предобработку:
# • Приведите тексты к нижнему регистру.
# • Замените все, кроме букв и цифр, на пробелы — это облегчит
# дальнейшее разделение текста на слова. Для такой замены в
# строке text подходит следующий вызов:
# re.sub('[^a-zA-Z0-9]', ' ', text)
data_train['FullDescription'].replace('[^a-zA-Z0-9]', ' ',
                                      regex = True,inplace=True)
data_test['FullDescription'].replace('[^a-zA-Z0-9]', ' ',
                                      regex = True,inplace=True)

# • Примените TfidfVectorizer для преобразования текстов в векторы
# признаков. Оставьте только те слова, которые встречаются хотя 
# бы в 5 объектах (параметр min_df у TfidfVectorizer).
vectorizer = TfidfVectorizer(min_df = 5)
fullDescription_train=vectorizer.fit_transform(data_train['FullDescription'])
fullDescription_test=vectorizer.transform(data_test['FullDescription'])

# • Замените пропуски в столбцах LocationNormalized и ContractTime 
# на специальную строку ’nan’. Код для этого был приведен выше.
data_train['LocationNormalized'].fillna('nan', inplace=True)
data_train['ContractTime'].fillna('nan', inplace=True)

# • Примените DictVectorizer для получения one-hot-кодирования
# признаков LocationNormalized и ContractTime.
enc = DictVectorizer()
X_train_categ = enc.fit_transform(data_train[['LocationNormalized', 
                                              'ContractTime']].to_dict('records'))
X_test_categ = enc.transform(data_test[['LocationNormalized', 
                                              'ContractTime']].to_dict('records'))

# • Объедините все полученные признаки в одну матрицу "объектыпризнаки". 
# Обратите внимание, что матрицы для текстов и
# категориальных признаков являются разреженными. 
# Для объединения их столбцов нужно воспользоваться
# функцией scipy.sparse.hstack
joinedMatrix_train = hstack([fullDescription_train, X_train_categ])


In [33]:
# Обучите гребневую регрессию с параметром alpha=1. 
# Целевая переменная записана в столбце SalaryNormalized.
clf = Ridge(alpha=1.0,random_state=241)
clf.fit(joinedMatrix_train, data_train['SalaryNormalized'])

Ridge(random_state=241)

In [34]:
# Постройте прогнозы для двух примеров из файла salary-test-mini.csv.
# Значения полученных прогнозов являются ответом на задание. 
# Укажите их через пробел.
joinedMatrix_test=hstack([fullDescription_test,X_test_categ])
predict=clf.predict(joinedMatrix_test)

In [35]:
print('Прогноз по зарплате')
print (' '.join(map(lambda x: str(round(x,2)), predict)))

Прогноз по зарплате
56574.07 37139.25
